In [2]:
import numpy as np
import pandas as pd

def as_colvec(x):
    if (x.ndim == 2):
        return x
    else:
        return np.expand_dims(x, axis=1)
    
def implied_returns(delta, sigma, w):
    """
Obtain the implied expected returns by reverse engineering the weights
Inputs:
delta: Risk Aversion Coefficient (scalar)
sigma: Variance-Covariance Matrix (N x N) as DataFrame
    w: Portfolio weights (N x 1) as Series
Returns an N x 1 vector of Returns as Series
    """
    ir = delta * sigma.dot(w).squeeze() # to get a series from a 1-column dataframe
    ir.name = 'Implied Returns'
    return ir

# Assumes that Omega is proportional to the variance of the prior
def proportional_prior(sigma, tau, p):
    """
    Returns the He-Litterman simplified Omega
    Inputs:
    sigma: N x N Covariance Matrix as DataFrame
    tau: a scalar
    p: a K x N DataFrame linking Q and Assets
    returns a P x P DataFrame, a Matrix representing Prior Uncertainties
    """
    helit_omega = p.dot(tau * sigma).dot(p.T)
    # Make a diag matrix from the diag elements of Omega
    return pd.DataFrame(np.diag(np.diag(helit_omega.values)),index=p.index, columns=p.index)

from numpy.linalg import inv

def bl(w_prior, sigma_prior, p, q,
                omega=None,
                delta=2.5, tau=.02):
    """
# Computes the posterior expected returns based on 
# the original black litterman reference model
#
# W.prior must be an N x 1 vector of weights, a Series
# Sigma.prior is an N x N covariance matrix, a DataFrame
# P must be a K x N matrix linking Q and the Assets, a DataFrame
# Q must be an K x 1 vector of views, a Series
# Omega must be a K x K matrix a DataFrame, or None
# if Omega is None, we assume it is
#    proportional to variance of the prior
# delta and tau are scalars
    """
    if omega is None:
        omega = proportional_prior(sigma_prior, tau, p)
    # Force w.prior and Q to be column vectors
    # How many assets do we have?
    N = w_prior.shape[0]
    # And how many views?
    K = q.shape[0]
    # First, reverse-engineer the weights to get pi
    pi = implied_returns(delta, sigma_prior,  w_prior)
    # Adjust (scale) Sigma by the uncertainty scaling factor
    sigma_prior_scaled = tau * sigma_prior  
    # posterior estimate of the mean, use the "Master Formula"
    # we use the versions that do not require
    # Omega to be inverted (see previous section)
    # this is easier to read if we use '@' for matrixmult instead of .dot()
    #     mu_bl = pi + sigma_prior_scaled @ p.T @ inv(p @ sigma_prior_scaled @ p.T + omega) @ (q - p @ pi)
    mu_bl = pi + sigma_prior_scaled.dot(p.T).dot(inv(p.dot(sigma_prior_scaled).dot(p.T) + omega).dot(q - p.dot(pi).values))
    # posterior estimate of uncertainty of mu.bl
#     sigma_bl = sigma_prior + sigma_prior_scaled - sigma_prior_scaled @ p.T @ inv(p @ sigma_prior_scaled @ p.T + omega) @ p @ sigma_prior_scaled
    sigma_bl = sigma_prior + sigma_prior_scaled - sigma_prior_scaled.dot(p.T).dot(inv(p.dot(sigma_prior_scaled).dot(p.T) + omega)).dot(p).dot(sigma_prior_scaled)
    return (mu_bl, sigma_bl)

# for convenience and readability, define the inverse of a dataframe
def inverse(d):
    """
    Invert the dataframe by inverting the underlying matrix
    """
    return pd.DataFrame(inv(d.values), index=d.columns, columns=d.index)

def w_msr(sigma, mu, scale=True):
    """
    Optimal (Tangent/Max Sharpe Ratio) Portfolio weights
    by using the Markowitz Optimization Procedure
    Mu is the vector of Excess expected Returns
    Sigma must be an N x N matrix as a DataFrame and Mu a column vector as a Series
    This implements page 188 Equation 5.2.28 of
    "The econometrics of financial markets" Campbell, Lo and Mackinlay.
    """
    w = inverse(sigma).dot(mu)
    if scale:
        w = w/sum(w) # fix: this assumes all w is +ve
    return w



In [9]:
import numpy as np
import pandas as pd

import edhec_risk_kit_206 as erk

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
ind_rets = erk.get_ind_returns(n_inds = 49)['2013':'2018'][['Hlth', 'Fin', 'Whlsl', 'Rtail', 'Food']]
mkt_caps = erk.get_ind_market_caps(n_inds = 49, weights = True)['2013':'2018'][['Hlth', 'Fin', 'Whlsl', 'Rtail', 'Food']]
mkt_caps.tail()

,Hlth,Fin,Whlsl,Rtail,Food
2018-08,0.005456,0.024948,0.010987,0.076225,0.013758
2018-09,0.005577,0.024080,0.010992,0.080234,0.013394
2018-10,0.005383,0.023262,0.010681,0.080886,0.012858
2018-11,0.005370,0.024093,0.010674,0.078849,0.014076
2018-12,0.005481,0.023729,0.010849,0.077102,0.013871


In [11]:
ind_vol = ind_rets.std()*np.sqrt(12)
ind_corr = ind_rets.corr()
ind_corr

,Hlth,Fin,Whlsl,Rtail,Food
Hlth,1.000000,0.524886,0.650900,0.574227,0.371698
Fin,0.524886,1.000000,0.777653,0.595332,0.343016
Whlsl,0.650900,0.777653,1.000000,0.743754,0.541244
Rtail,0.574227,0.595332,0.743754,1.000000,0.575063
Food,0.371698,0.343016,0.541244,0.575063,1.000000


In [12]:
ind_ann_cov = ind_vol.dot(ind_vol) * ind_corr
ind_ann_cov

,Hlth,Fin,Whlsl,Rtail,Food
Hlth,0.108718,0.057064,0.070764,0.062429,0.040410
Fin,0.057064,0.108718,0.084545,0.064723,0.037292
Whlsl,0.070764,0.084545,0.108718,0.080859,0.058843
Rtail,0.062429,0.064723,0.080859,0.108718,0.062519
Food,0.040410,0.037292,0.058843,0.062519,0.108718


In [15]:
cw = erk.weight_cw(ind_rets, cap_weights = mkt_caps)
delta = 2.5
implied_returns(delta, ind_ann_cov, cw)

Hlth     0.152854
Fin      0.176483
Whlsl    0.201786
Rtail    0.223770
Food     0.157528
Name: Implied Returns, dtype: float64

In [16]:
cw

Hlth     0.042624
Fin      0.184533
Whlsl    0.096043
Rtail    0.537344
Food     0.139456
Name: 2013-02, dtype: float64

In [17]:
q = pd.Series([.03]) # just one view
# start with a single view, all zeros and overwrite the specific view
p = pd.DataFrame([0.]*len(ind_rets.columns), index=ind_rets.columns).T
w_Rtail =  cw.loc["Rtail"]/(cw.loc["Rtail"]+cw.loc["Whlsl"])
w_Whlsl =  cw.loc["Whlsl"]/(cw.loc["Rtail"]+cw.loc["Whlsl"])
p.iloc[0]['Hlth'] = 1.
p.iloc[0]['Rtail'] = -w_Rtail
p.iloc[0]['Whlsl'] = -w_Whlsl
p

,Hlth,Fin,Whlsl,Rtail,Food
0,1.0,0.0,-0.151635,-0.848365,0.0


In [18]:
delta = 2.5
tau = 0.05

In [22]:
rho = ind_rets.corr()
vols = ind_rets.std()*np.sqrt(12)
sigma_prior = vols.dot(vols)*rho

In [23]:
sigma_prior

,Hlth,Fin,Whlsl,Rtail,Food
Hlth,0.108718,0.057064,0.070764,0.062429,0.040410
Fin,0.057064,0.108718,0.084545,0.064723,0.037292
Whlsl,0.070764,0.084545,0.108718,0.080859,0.058843
Rtail,0.062429,0.064723,0.080859,0.108718,0.062519
Food,0.040410,0.037292,0.058843,0.062519,0.108718


In [24]:
bl_mu, bl_sigma = bl(cw, sigma_prior, p, q, tau = tau)
(bl_mu*100).round(3)

Hlth     17.936
Fin      17.021
Whlsl    19.336
Rtail    19.901
Food     14.484
dtype: float64

In [25]:
wmsr = w_msr(sigma=bl_sigma, mu=bl_mu)
# display w*
(wmsr*100).round(1)

Hlth     27.6
Fin      18.5
Whlsl     6.1
Rtail    33.9
Food     13.9
dtype: float64

In [26]:
q = pd.Series([.05]) # just one view
# start with a single view, all zeros and overwrite the specific view
p = pd.DataFrame([0.]*len(ind_rets.columns), index=ind_rets.columns).T
w_Rtail =  cw.loc["Rtail"]/(cw.loc["Rtail"]+cw.loc["Whlsl"])
w_Whlsl =  cw.loc["Whlsl"]/(cw.loc["Rtail"]+cw.loc["Whlsl"])
p.iloc[0]['Hlth'] = 1.
p.iloc[0]['Rtail'] = -w_Rtail
p.iloc[0]['Whlsl'] = -w_Whlsl
p

,Hlth,Fin,Whlsl,Rtail,Food
0,1.0,0.0,-0.151635,-0.848365,0.0


In [27]:
bl_mu, bl_sigma = bl(cw, sigma_prior, p, q, tau = tau)
(bl_mu*100).round(3)

Hlth     18.479
Fin      16.892
Whlsl    19.163
Rtail    19.393
Food     14.224
dtype: float64

In [28]:
wmsr = w_msr(sigma=bl_sigma, mu=bl_mu)
# display w*
(wmsr*100).round(1)

Hlth     32.5
Fin      18.5
Whlsl     5.3
Rtail    29.8
Food     13.9
dtype: float64